In [46]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re
import pytesseract
import pandas as pd


pathname0 = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/Metropolitan_DA/Line'
pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/Metropolitan_DA/Line0'


def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def listdir_nohidden1(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
class Convert_to_line:
    def __init__(self,im):
        self.im=im
    
    def get_concat_h(self,im1, im2):
        dst = Image.new('RGB', (im1.width + im2.width, im1.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (im1.width, 0))
        return dst

    def crop(self):
        #Top Crop
        width,height=self.im.size
        top=0.2*height
        bottom=0.8*height

        left=0.1*width
        right=1*width
        IM1=self.im.crop((left,top,right,bottom))
        return IM1

def cjk_detect(texts):
    # korean
    if re.search("[\uac00-\ud7a3]", texts):
        return "ko"
    # japanese
    if re.search("[\u3040-\u30ff]", texts):
        return "ja"
    # chinese
    if re.search("[\u4e00-\u9FFF]", texts):
        return "zh"
    return None

pattern = re.compile(
    "\u3041-\u3096" # Hiragana
    "\u30A0-\u30FF" # Katakana
)


def Count(text):
    i=0
    num=0
    text = text.replace('\n',"")
    for row in text:
        if "子" in row:
            i=i+1        
            num=num+1
        if cjk_detect(row) == "ja":
            if cjk_detect(text[i+1]) == "ja":
                i=i+1
                num=num+1
            else:
                i=i+1
    return(num)

In [47]:
#Step2
#Convert to line
from PIL import Image
import os, cv2

def listdir_nohidden1(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
            
for i in range(0,len(sorted(listdir_nohidden1(pathname0)))):
    image = sorted(listdir_nohidden1(pathname0))[i]
    image_test = Image.open(pathname0+"/"+image)
    
    test=Convert_to_line(image_test)
    line=test.crop() #Remove redundant space around image
    
    quality_val = 90
    line.save(pathname+"/"+image, 'JPEG', quality=quality_val)

In [48]:
import cv2
#Detecting circles
img = cv2.imread(pathname+"/"+image, cv2.IMREAD_COLOR)

# Detecting circles: This step is heavily refering to the following website https://www.geeksforgeeks.org/circle-detection-using-opencv-python/
# Convert to grayscale.
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))  

# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 150, param1 = 50,
               param2 = 15, minRadius = 8, maxRadius = 12)
detected_circles = np.uint16(np.around(detected_circles))

In [52]:
#Crop into departments
circle_list=sorted(detected_circles[0], key=lambda x: x[0])
im=img
pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/Metropolitan_DA/Departments'

for i in range(0,len(circle_list)+1):
    try:        
        if i ==0:
            dept = im[0:im.shape[0], 0:circle_list[0][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png", quality=quality_val)
            i=i+1
        if i>0 and len(str(i))==1:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png", quality=quality_val)
            i=i+1
        if i>0 and len(str(i))==2:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept0"+str(i)+".png", quality=quality_val)
            i=i+1
        if i ==len(detected_circles1[0])+1:
            dept = im[0:im.shape[0], circle_list[i][0]+15:im.shape[1]]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png", quality=quality_val)
            i=i+1
        else:
            dept = im[0:im.shape[0], circle_list[i-1][0]+15:circle_list[i][0]+15]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png", quality=quality_val)
            i=i+1
    except:
        i=i+1
    


In [53]:
#Erase miss-detected departments
indexes = [0,6]
circle_list = [circle_list[i] for i in indexes]

for i in range(0,len(circle_list)+1):
    try:        
        if i ==0:
            dept = im[0:im.shape[0], 0:circle_list[0][0]+30]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==1:
            dept = im[0:im.shape[0], circle_list[i-1][0]+30:circle_list[i][0]+30]
            Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
            i=i+1
        if i>0 and len(str(i))==2:
            dept = im[0:im.shape[0], circle_list[i-1][0]+30:circle_list[i][0]+30]
            Image.fromarray(dept).save(pathname+"/Dept0"+str(i)+".png")
            i=i+1
        if i ==len(detected_circles1[0])+1:
            dept = im[0:im.shape[0], circle_list[i][0]+30:im.shape[1]]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
        else:
            dept = im[0:im.shape[0], circle_list[i-1][0]+30:circle_list[i][0]+30]
            Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")
            i=i+1
    except:
        i=i+1


In [58]:
from PIL import Image
import pytesseract

pytesseract.pytesseract.tesseract_cmd = 'C:/OCR/Tesseract-OCR/tesseract.exe'  # your path may be different

file = Image.open(pathname+"/Dept00"+str(0)+".png")
str = pytesseract.image_to_string(file , lang="jpn")

print(str)

TesseractNotFoundError: C:/OCR/Tesseract-OCR/tesseract.exe is not installed or it's not in your PATH. See README file for more information.

In [56]:
file.show()